In [1]:
from imports import *
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
%matplotlib inline

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [25]:
def read_NASA_stellar_data():
    d = np.genfromtxt('input_data/NASAexoplanet_Aug222018_stellarv4.csv', skip_header=50, delimiter=',')
    print d.shape
    ids,_,_,_,ra,_,dec,dist,udist,ldist,_,Teff,uTeff,lTeff,_,Ms,uMs,lMs,_,Rs,uRs,lRs,_,_,ls,bs,Ls,uLs,lLs,_,Vmag,eVmag,_,Jmag,eJmag,_,Hmag,eHmag,_,Kmag,eKmag,_ = d.T
    _,names = np.genfromtxt('input_data/NASAexoplanet_Aug222018_stellarv4.csv', skip_header=51, delimiter=',', usecols=(0,1), dtype='|S30').T
    return ids, names, ra, dec, ls, bs, dist, udist, ldist, Teff, uTeff, lTeff, Ms, uMs, lMs, Rs, uRs, lRs, Ls, uLs, lLs, Vmag, eVmag, Jmag, eJmag, Hmag, eHmag, Kmag, eKmag

In [26]:
# get data
ids, names, ra, dec, ls, bs, dist, udist, ldist, Teff, uTeff, lTeff, Ms, uMs, lMs, Rs, uRs, lRs, Ls, uLs, lLs, Vmag, eVmag, JVmag, eJmag, Hmag, eHmag, Kmag, eKmag = read_NASA_stellar_data()
print 'Nentries =', ids.size

(459, 42)
Nentries = 459


In [27]:
def query_gaia(ra_deg, dec_deg, rad_arcsec=5):
    coord = SkyCoord(ra=ra_deg, dec=dec_deg, unit=(u.degree, u.degree), frame='icrs')
    r = Gaia.query_object_async(coordinate=coord, radius=u.Quantity(rad_arcsec, u.arcsecond))
    if len(r) > 0:
        i = np.where(r['dist'].base == np.min(r['dist'].base))[0][0]
    else:
        raise ValueError('No solutions. radius = %.1f arcsec'%rad_arcsec)
    ra, dec, par, epar = r[i]['ra'], r[i]['dec'], r[i]['parallax'], r[i]['parallax_error']
    return ra, dec, par, epar

In [28]:
# query gaia for each star
#_,inds = np.unique(names, return_index=True)
results = np.zeros((ids.size,5)) + np.nan
results[:,0] = ids
for i in range(ids.size):
    print float(i)/ids.size
    try:
        results[i,1:] = query_gaia(ra[i], dec[i], 10)
    except ValueError:
        pass

0.0
Query finished.
0.00217864923747
Query finished.
0.00435729847495
Query finished.
0.00653594771242
Query finished.
0.00871459694989
Query finished.
0.0108932461874
Query finished.
0.0130718954248
Query finished.
0.0152505446623
Query finished.
0.0174291938998
Query finished.
0.0196078431373
Query finished.
0.0217864923747
Query finished.
0.0239651416122
Query finished.
0.0261437908497
Query finished.
0.0283224400871
Query finished.
0.0305010893246
Query finished.
0.0326797385621
Query finished.
0.0348583877996
Query finished.
0.037037037037


/Users/ryancloutier/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Warning: converting a masked element to nan.


Query finished.
0.0392156862745
Query finished.
0.041394335512
Query finished.
0.0435729847495
Query finished.
0.0457516339869
Query finished.
0.0479302832244
Query finished.
0.0501089324619
Query finished.
0.0522875816993
Query finished.
0.0544662309368
Query finished.
0.0566448801743
Query finished.
0.0588235294118
Query finished.
0.0610021786492
Query finished.
0.0631808278867
Query finished.
0.0653594771242
Query finished.
0.0675381263617
Query finished.
0.0697167755991
Query finished.
0.0718954248366
Query finished.
0.0740740740741
Query finished.
0.0762527233115
Query finished.
0.078431372549
Query finished.
0.0806100217865
Query finished.
0.082788671024
Query finished.
0.0849673202614
Query finished.
0.0871459694989
Query finished.
0.0893246187364
Query finished.
0.0915032679739
Query finished.
0.0936819172113
Query finished.
0.0958605664488
Query finished.
0.0980392156863
Query finished.
0.100217864924
Query finished.
0.102396514161
Query finished.
0.104575163399
Query finished

Query finished.
0.616557734205
Query finished.
0.618736383442
Query finished.
0.62091503268
Query finished.
0.623093681917
Query finished.
0.625272331155
Query finished.
0.627450980392
Query finished.
0.62962962963
Query finished.
0.631808278867
Query finished.
0.633986928105
Query finished.
0.636165577342
Query finished.
0.63834422658
Query finished.
0.640522875817
Query finished.
0.642701525054
Query finished.
0.644880174292
Query finished.
0.647058823529
Query finished.
0.649237472767
Query finished.
0.651416122004
Query finished.
0.653594771242
Query finished.
0.655773420479
Query finished.
0.657952069717
Query finished.
0.660130718954
Query finished.
0.662309368192
Query finished.
0.664488017429
Query finished.
0.666666666667
Query finished.
0.668845315904
Query finished.
0.671023965142
Query finished.
0.673202614379
Query finished.
0.675381263617
Query finished.
0.677559912854
Query finished.
0.679738562092
Query finished.
0.681917211329
Query finished.
0.684095860566
Query finis

In [29]:
# save results
hdr = 'ID, RA [deg], Dec [deg], Parallax [mas], eParallax [mas]'
np.savetxt('input_data/GAIA_query_resultsv4.csv', results, delimiter=',', fmt='%.15f', header=hdr)

## old stuff

In [8]:
# create query form
_,inds = np.unique(names, return_index=True)
names2, ra2, dec2 = names[inds], ra[inds], dec[inds]
g1, g2 = '', ''
for i in range(names2.size):
    label = '+' if dec2[i] >= 0 else ''
    g1 += '%s\t%.7f%s%.7f (degrees, degrees)\n'%(names2[i], ra2[i], label, dec2[i])
    g1 += '%.7f %.7f\n'%(ra2[i], dec2[i])
    g2 += '%s\n'%names2[i]
    
f = open('input_data/gaia_query.txt', 'w')
f.write(g2)
f.close()

f = open('input_data/gaia_query_crosscheck.txt', 'w')
f.write(g1)
f.close()

In [9]:
print g1

EPIC 211822797	130.4103700+17.6400050 (degrees, degrees)
130.4103700 17.6400050
GJ 1132	153.7157370-47.1567270 (degrees, degrees)
153.7157370 -47.1567270
GJ 1214	258.8288370+4.9639200 (degrees, degrees)
258.8288370 4.9639200
GJ 3470	119.7743490+15.3914920 (degrees, degrees)
119.7743490 15.3914920
GJ 9827	351.7701420-1.2862730 (degrees, degrees)
351.7701420 -1.2862730
HAT-P-54	99.8980030+25.4825400 (degrees, degrees)
99.8980030 25.4825400
HATS-6	88.1467970-19.0316560 (degrees, degrees)
88.1467970 -19.0316560
K2-104	129.6367490+19.7738290 (degrees, degrees)
129.6367490 19.7738290
K2-116	336.1516110-11.5786910 (degrees, degrees)
336.1516110 -11.5786910
K2-117	133.8556820+10.4691310 (degrees, degrees)
133.8556820 10.4691310
K2-120	132.1429900+17.1906850 (degrees, degrees)
132.1429900 17.1906850
K2-122	126.4763110+20.3595700 (degrees, degrees)
126.4763110 20.3595700
K2-123	134.4441830+21.4535330 (degrees, degrees)
134.4441830 21.4535330
K2-124	133.6411900+23.1328910 (degrees, degrees)
133.6